In [1]:
"""
5-3 순환신경망 활용

#임베딩
컴퓨터는 우리가 사용하는 한글, 영어 등을 그대로 이해하지 못한다 
컴퓨터가 이해하는 벡터로 변경해야 하는데 그것을 임베딩 이라고 말한다

크게 단어 수준의 임베딩 - word2Vec, FastText
문장 수준의 임베딩 - Elmo, Berr, GPT 등으로 구분할 수 있다
단어 수준의 임베딩은 동음이의어를 구별할 수 없지만, 
문장 수준의 임베딩은 사람처럼 문장의 앞뒤를 보고 이를 파악할수 있다
 
단어 수준의 임베딩을 위해 단어, 음절, 형태소 등으로 나누고 이를 수치로 변환한다
가장 간단한 벡터 변환 방법은 원핫 인코딩

이 방법은 매우 간단하지만 단어가 많아질수록 벡터의 공간이 커지고 비효율이 발생하는 문제가 있다
원핫 인코딩은 행렬 값이 대부분0 인 희소행렬(sparse matrix) 이라고도 부른다
엄청난 크기의 희소행렬은 메모리 낭비가 심하고, 또한 원핫 인코딩은 단어와 단어의 유사도를 
알기 어렵다는 한계가 있다
반대로 단어를 밀집행렬로 표현할 수도 있는데, 이때 행렬은 0이 아닌 실수값으로 채워진다
밀집행렬 형태는 단어와 단어의 유사도를 알 수 있고,  자원의 낭비가 작다는 장점이 있다
텐서플로에서는 임베징 레이어를 제공하고 있는데, 희소행렬이 아닌 밀집행렬로 구현되어 있다

임베딩 레이어에는 두가지 파라미터 값이 필요하다. 첫 번째 입력 차원(단어의 총 수)이고
두번째는 임베딩 차원이다


임의의 4개의 숫자 12, 8, 15, 20를 임베딩 레이어에 통과시키면 각각 원소 3개를 갖는 벡터로 변환되어 
밀집행렬로 정리되는 것을 알 수 있다
"""
# 라이브러리 불러오기
import tensorflow as tf

# 임베딩 레이어
embedding_layer = tf.keras.layers.Embedding(100, 3)
result = embedding_layer(tf.constant([12, 8, 15, 20])) # 더미 데이터 입력
print(result)

tf.Tensor(
[[ 0.02298846 -0.00393497  0.02694971]
 [-0.04439829  0.03345111  0.04640342]
 [ 0.01744829  0.03253639  0.01392256]
 [-0.0277378   0.03973353  0.04665048]], shape=(4, 3), dtype=float32)


In [2]:
"""
임베딩 레이어는 input_dim, output_dim, input_length 파라미터가 있다
input_dim 은 단어 사전의 크기를 의미한다 
output_dim은 출력 벡터의 크기를 말한다 
다음 예제에서는 output_dim으로 3을 지정했다

임베딩 레이어를 모델에 적용해보자
일반적으로 자연어 처리 모델에서는 데이터를 입력하는 부분에 임베딩 레이어를 활용한다
또한, 모델에 따라 입력 길이를 설정해야 할 수도 있고 그렇지 않을 수도 있다
다음은 입력길이를 32로 설정하는 예제
"""
# 임베딩 레이어 활용
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(100, 3, input_length=32))
model.add(tf.keras.layers.LSTM(units=32))
model.add(tf.keras.layers.Dense(units=1))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 32, 3)             300       
                                                                 
 lstm (LSTM)                 (None, 32)                4608      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 4,941
Trainable params: 4,941
Non-trainable params: 0
_________________________________________________________________


In [3]:
"""
라이브러리 불러오기 - 위에 작성한 같은 모델
"""
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 임베딩 레이어 활용
model = Sequential()
model.add(Embedding(100, 3, input_length =32))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 32, 3)             300       
                                                                 
 lstm_1 (LSTM)               (None, 32)                4608      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,941
Trainable params: 4,941
Non-trainable params: 0
_________________________________________________________________


In [4]:
"""
양방향(Bidirectional) RNN
자연어 데이터의 경우 순서대로 데이터를 처리하고, 역순으로 처리할 경우 더 좋은 성능을 
발휘하기도 한다
텐서플로에서는 tensorflow.keras.layers에 양방향 레이어(Bidirectional)를 제공하고 있고
RNN 레이어를 감싸면 쉽게 양방향 RNN을 만들 수 있다
- 예시 ) 나는 ㅁㅁㅁ를 보고 싶다
앞에 나온 "나는" 이란 단어만 가지고는 예측할 수 없지만 뒤에 나오는 "보고싶다" 라는
단어를 바탕으로 다양한 단어를 추측할 수 있기 때문이다


모델의 summary() 결과에서는 LSTM(units=32)과 같이 유닛의 개수를 32로 지정해서
모델을 만들었지만 유닛의 개수가 32의 2배인 64개가 된다
순방향과 역방향으로 각각 LSTM 모델이 만들어졌기 때문이다
이를 Bidirectional LSTM 이라고 부르며 줄여서 Bi-LSTM이라고 표현하기도 한다
"""
#Bidirectional LSTM
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(100, 3))
model.add(Bidirectional(LSTM(32))) # 양방향 RNN
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 3)           300       
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9216      
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9,581
Trainable params: 9,581
Non-trainable params: 0
_________________________________________________________________


In [5]:
"""
스태킹 RNN
레이어를 쌓는다 
레이어를 쌓기 위해서는 다른 층(레이어)의 출력도 허용해야한다 
RNN 레이어 파라미터 return_sequences 를 True로 설정한다 
만약 return_sequences 를 기본값(False)으로 그대로 두면 마지막 상태값만 출력되어 
RNN 레이어를 쌓을 수 없다 - 즉 에러 발생

참고로 최상단에 있는 RNN 레이어는 더 이상 모든 상태를 전달할 필요가 없어
return_sequences 설정을 True로 설정할 필요가 없다
"""
#스태킹 RNN 예제
model = Sequential()
model.add(Embedding(100, 32))
model.add(LSTM(32, return_sequences=True)) # 전체 시퀀스 출력
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          3200      
                                                                 
 lstm_3 (LSTM)               (None, None, 32)          8320      
                                                                 
 lstm_4 (LSTM)               (None, 32)                8320      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 19,873
Trainable params: 19,873
Non-trainable params: 0
_________________________________________________________________


In [7]:
"""
순환 드롭아웃

내장 RNN은 기본적으로 텐서플로 레이어에서 제공하는 유용한 기능인 dropout 외에 
recurrent_dropout을 제공하고 있다. dropout은 입력에 대한 드롭아웃 비율이고
recurrent_dropout은 순환 상태의 드롭아웃 비율을 뜻한다 
모두 0과 1 사이의 부동 소수점이고 과대 적합을 방지하기 위해 내장된 기능이다
"""
# 순환 드롭아웃
model = Sequential()
model.add(Embedding(100, 32))
model.add(LSTM(32, recurrent_dropout=0.2, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 32)          3200      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 11,553
Trainable params: 11,553
Non-trainable params: 0
_________________________________________________________________
